# 🤖 Notebook 3 : Entraînement des Modèles XGBoost

## Objectif : Entraîner et optimiser les modèles pour identifier les opportunités de substitution aux importations

### Ce notebook couvre :
1. Chargement des données préparées (feature engineering)
2. Définition des cibles de prédiction
3. Préparation des données pour l'entraînement
4. Entraînement de modèles XGBoost multi-cibles
5. Optimisation des hyperparamètres
6. Évaluation des performances
7. Analyse de l'importance des features
8. Sauvegarde des modèles entraînés

### Modèles à entraîner :
- **Modèle 1** : Prédiction du potentiel de production (production_fcfa)
- **Modèle 2** : Score de substitution aux importations (ratio production/imports)
- **Modèle 3** : Classification des opportunités (haute/moyenne/faible)

## 1. 📦 Import des bibliothèques

In [1]:
# Bibliothèques de base
import pandas as pd
import numpy as np
from pathlib import Path
import json
import pickle
import warnings
warnings.filterwarnings('ignore')

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import (mean_squared_error, mean_absolute_error, r2_score,
                            accuracy_score, classification_report, confusion_matrix,
                            precision_score, recall_score, f1_score)

# XGBoost
import xgboost as xgb
from xgboost import XGBRegressor, XGBClassifier

# Configuration
plt.style.use('seaborn-v0_8-whitegrid')
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

print("✅ Bibliothèques importées avec succès!")
print(f"📦 XGBoost version: {xgb.__version__}")

✅ Bibliothèques importées avec succès!
📦 XGBoost version: 3.1.2


## 2. 📂 Chargement des données préparées

In [2]:
# Chemins des données
DATA_PATH = Path("../data/processed")
MODELS_PATH = Path("../models")
MODELS_PATH.mkdir(exist_ok=True)

# Charger le dataset ML-ready
df = pd.read_csv(DATA_PATH / "dataset_ml_complete.csv")

# Charger les métadonnées et encodeurs
with open(DATA_PATH / "feature_metadata.json", 'r', encoding='utf-8') as f:
    feature_metadata = json.load(f)

with open(DATA_PATH / "encoders.pkl", 'rb') as f:
    encoders = pickle.load(f)

print("✅ Données chargées avec succès!")
print(f"\n📊 Dimensions du dataset: {df.shape[0]} lignes × {df.shape[1]} colonnes")
print(f"📊 Nombre de features numériques: {feature_metadata['nb_features']}")
print(f"\n📋 Secteurs disponibles: {df['LIBELLES'].nunique()}")
print(f"📅 Période: {df['year'].min()} - {df['year'].max()}")

df.head()

✅ Données chargées avec succès!

📊 Dimensions du dataset: 210 lignes × 52 colonnes
📊 Nombre de features numériques: 48

📋 Secteurs disponibles: 21
📅 Période: 2014 - 2023


,SH,LIBELLES,year,production_fcfa,production_tonnes,consommation_fcfa,consommation_tonnes,imports_fcfa,exports_fcfa,imports_tonnes,exports_tonnes,year_normalized,years_since_start,economic_period,is_covid_period,balance_commerciale_fcfa,balance_commerciale_tonnes,taux_couverture,ratio_prod_conso_fcfa,ratio_prod_imports,intensite_export,prix_unitaire_production,prix_unitaire_imports,prix_unitaire_exports,total_production_fcfa,total_production_tonnes,part_marche_production,indice_specialisation,valeur_ajoutee_estimee,demande_interieure,taux_autosuffisance,production_fcfa_lag1,production_fcfa_lag2,production_tonnes_lag1,production_tonnes_lag2,imports_fcfa_lag1,imports_fcfa_lag2,exports_fcfa_lag1,exports_fcfa_lag2,production_fcfa_growth,production_tonnes_growth,imports_fcfa_growth,exports_fcfa_growth,production_fcfa_ma3,production_tonnes_ma3,imports_fcfa_ma3,exports_fcfa_ma3,production_trend,sector_encoded,period_encoded,sector_size_category,sector_size_encoded
0,I,Animaux vivants et produits du règne animal,2014,24.500,100.600,14.900,79.000,19.700,4.800,89.800,10.800,0.000,0,pre_2017,0,-14.900,-79.000,24.365,1.644,1.244,19.592,0.244,0.219,0.444,3056.100,5761.500,0.802,126.744,0.225,39.400,62.183,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,24.500,100.600,19.700,4.800,2.416,0,3,small,2
1,I,Animaux vivants et produits du règne animal,2015,21.900,128.300,15.900,74.500,18.900,3.000,101.400,26.900,0.111,1,pre_2017,0,-15.900,-74.500,15.873,1.377,1.159,13.699,0.171,0.186,0.112,3126.400,6266.000,0.700,110.746,-0.075,37.800,57.937,24.500,0.000,100.600,0.000,19.700,0.000,4.800,0.000,-10.612,27.535,-4.061,-37.500,23.200,114.450,19.300,3.900,2.416,0,3,small,2
2,I,Animaux vivants et produits du règne animal,2016,19.800,98.700,15.800,90.700,17.800,2.000,94.700,4.000,0.222,2,pre_2017,0,-15.800,-90.700,11.236,1.253,1.112,10.101,0.201,0.188,0.500,3483.800,7737.000,0.568,89.855,0.312,35.600,55.618,21.900,24.500,128.300,100.600,18.900,19.700,3.000,4.800,-9.589,-23.071,-5.820,-33.333,22.067,109.200,18.800,3.267,2.416,0,3,small,2
3,I,Animaux vivants et produits du règne animal,2017,22.100,121.100,17.300,114.700,19.700,2.400,117.900,3.200,0.333,3,2017_2019,0,-17.300,-114.700,12.183,1.277,1.122,10.860,0.182,0.167,0.750,3979.100,7398.300,0.555,87.808,0.583,39.400,56.091,19.800,21.900,98.700,128.300,17.800,18.900,2.000,3.000,11.616,22.695,10.674,20.000,21.267,116.033,18.800,2.467,2.416,0,0,small,2
4,I,Animaux vivants et produits du règne animal,2018,25.300,136.700,22.900,129.100,24.100,1.200,132.900,3.800,0.444,4,2017_2019,0,-22.900,-129.100,4.979,1.105,1.050,4.743,0.185,0.181,0.316,4259.100,7462.100,0.594,93.914,0.134,48.200,52.490,22.100,19.800,121.100,98.700,19.700,17.800,2.400,2.000,14.480,12.882,22.335,-50.000,22.400,118.833,20.533,1.867,2.416,0,0,small,2


## 3. 🎯 Création des variables cibles pour la substitution aux importations

In [3]:
# Créer une copie de travail
df_ml = df.copy()

# ============================================================
# Variable cible 1: Score de substitution aux importations
# ============================================================
# Ce score mesure le potentiel de substitution = Production / Imports
# Plus le score est élevé, plus le secteur peut remplacer les importations

df_ml['substitution_score'] = np.where(
    df_ml['imports_fcfa'] > 0,
    df_ml['production_fcfa'] / df_ml['imports_fcfa'],
    df_ml['production_fcfa']  # Si pas d'imports, le score = production
)

# Normaliser le score entre 0 et 100
df_ml['substitution_score_normalized'] = (
    (df_ml['substitution_score'] - df_ml['substitution_score'].min()) / 
    (df_ml['substitution_score'].max() - df_ml['substitution_score'].min()) * 100
)

# ============================================================
# Variable cible 2: Classification des opportunités
# ============================================================
# Classifier les secteurs en 3 catégories d'opportunité

def classify_opportunity(row):
    """
    Classifie l'opportunité de substitution basée sur plusieurs critères:
    - Production élevée par rapport aux imports
    - Taux d'autosuffisance
    - Croissance de la production
    """
    score = 0
    
    # Critère 1: Ratio Production/Imports
    if row['ratio_prod_imports'] > 5:
        score += 3
    elif row['ratio_prod_imports'] > 1:
        score += 2
    elif row['ratio_prod_imports'] > 0.5:
        score += 1
    
    # Critère 2: Taux d'autosuffisance
    if row['taux_autosuffisance'] > 80:
        score += 3
    elif row['taux_autosuffisance'] > 50:
        score += 2
    elif row['taux_autosuffisance'] > 30:
        score += 1
    
    # Critère 3: Croissance de la production
    if row['production_fcfa_growth'] > 10:
        score += 2
    elif row['production_fcfa_growth'] > 0:
        score += 1
    
    # Classification finale
    if score >= 6:
        return 2  # Haute opportunité
    elif score >= 3:
        return 1  # Moyenne opportunité
    else:
        return 0  # Faible opportunité

df_ml['opportunity_class'] = df_ml.apply(classify_opportunity, axis=1)

# Labels pour les classes
opportunity_labels = {0: 'Faible', 1: 'Moyenne', 2: 'Haute'}
df_ml['opportunity_label'] = df_ml['opportunity_class'].map(opportunity_labels)

print("✅ Variables cibles créées!")
print(f"\n📊 Distribution du score de substitution:")
print(df_ml['substitution_score_normalized'].describe())

print(f"\n📊 Distribution des classes d'opportunité:")
print(df_ml['opportunity_label'].value_counts())

✅ Variables cibles créées!

📊 Distribution du score de substitution:
count   210.000
mean      2.626
std      12.865
min       0.000
25%       0.012
50%       0.012
75%       0.015
max     100.000
Name: substitution_score_normalized, dtype: float64

📊 Distribution des classes d'opportunité:
opportunity_label
Moyenne    109
Haute       85
Faible      16
Name: count, dtype: int64


In [4]:
# Visualisation de la distribution des opportunités
fig = make_subplots(rows=1, cols=2, 
                    subplot_titles=('Distribution du Score de Substitution', 
                                   'Classes d\'Opportunité par Secteur'))

# Histogramme du score
fig.add_trace(
    go.Histogram(x=df_ml['substitution_score_normalized'], 
                 nbinsx=30, 
                 marker_color='steelblue',
                 name='Score'),
    row=1, col=1
)

# Barplot des classes
class_counts = df_ml['opportunity_label'].value_counts()
colors = {'Faible': '#e74c3c', 'Moyenne': '#f39c12', 'Haute': '#27ae60'}
fig.add_trace(
    go.Bar(x=class_counts.index, 
           y=class_counts.values,
           marker_color=[colors[c] for c in class_counts.index],
           name='Classes'),
    row=1, col=2
)

fig.update_layout(
    title_text="🎯 Analyse des Opportunités de Substitution aux Importations",
    showlegend=False,
    height=400
)
fig.show()

## 4. 🔧 Préparation des données pour l'entraînement

In [5]:
# Définir les features à utiliser pour l'entraînement
# Exclure les identifiants, les variables cibles et les variables textuelles

exclude_from_features = [
    'SH', 'LIBELLES', 'economic_period', 'sector_size_category',
    'opportunity_label', 'opportunity_class', 
    'substitution_score', 'substitution_score_normalized',
    # Exclure aussi les variables qui sont directement utilisées pour calculer la cible
    'ratio_prod_imports', 'taux_autosuffisance'
]

# Features pour le modèle de régression (prédiction du score)
feature_cols_regression = [col for col in df_ml.columns 
                           if col not in exclude_from_features 
                           and df_ml[col].dtype in ['int64', 'float64', 'int32', 'float32']]

# Features pour le modèle de classification
feature_cols_classification = feature_cols_regression.copy()

print(f"📊 Nombre de features pour l'entraînement: {len(feature_cols_regression)}")
print(f"\n📋 Liste des features:")
for i, col in enumerate(feature_cols_regression, 1):
    print(f"   {i:2d}. {col}")

📊 Nombre de features pour l'entraînement: 46

📋 Liste des features:
    1. year
    2. production_fcfa
    3. production_tonnes
    4. consommation_fcfa
    5. consommation_tonnes
    6. imports_fcfa
    7. exports_fcfa
    8. imports_tonnes
    9. exports_tonnes
   10. year_normalized
   11. years_since_start
   12. is_covid_period
   13. balance_commerciale_fcfa
   14. balance_commerciale_tonnes
   15. taux_couverture
   16. ratio_prod_conso_fcfa
   17. intensite_export
   18. prix_unitaire_production
   19. prix_unitaire_imports
   20. prix_unitaire_exports
   21. total_production_fcfa
   22. total_production_tonnes
   23. part_marche_production
   24. indice_specialisation
   25. valeur_ajoutee_estimee
   26. demande_interieure
   27. production_fcfa_lag1
   28. production_fcfa_lag2
   29. production_tonnes_lag1
   30. production_tonnes_lag2
   31. imports_fcfa_lag1
   32. imports_fcfa_lag2
   33. exports_fcfa_lag1
   34. exports_fcfa_lag2
   35. production_fcfa_growth
   36. produ

In [6]:
# Préparation des données
X = df_ml[feature_cols_regression].values
y_regression = df_ml['substitution_score_normalized'].values
y_classification = df_ml['opportunity_class'].values

# Split temporel : utiliser les dernières années comme test
# Pour respecter la nature temporelle des données
train_years = [2014, 2015, 2016, 2017, 2018, 2019, 2020]
test_years = [2021, 2022, 2023]

train_mask = df_ml['year'].isin(train_years)
test_mask = df_ml['year'].isin(test_years)

X_train = df_ml.loc[train_mask, feature_cols_regression].values
X_test = df_ml.loc[test_mask, feature_cols_regression].values

y_train_reg = df_ml.loc[train_mask, 'substitution_score_normalized'].values
y_test_reg = df_ml.loc[test_mask, 'substitution_score_normalized'].values

y_train_clf = df_ml.loc[train_mask, 'opportunity_class'].values
y_test_clf = df_ml.loc[test_mask, 'opportunity_class'].values

# Normalisation des features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("✅ Données préparées pour l'entraînement!")
print(f"\n📊 Split temporel:")
print(f"   • Train: années {train_years} ({len(X_train)} échantillons)")
print(f"   • Test: années {test_years} ({len(X_test)} échantillons)")
print(f"\n📊 Distribution des classes (train):")
unique, counts = np.unique(y_train_clf, return_counts=True)
for u, c in zip(unique, counts):
    print(f"   • Classe {u} ({opportunity_labels[u]}): {c} ({c/len(y_train_clf)*100:.1f}%)")

✅ Données préparées pour l'entraînement!

📊 Split temporel:
   • Train: années [2014, 2015, 2016, 2017, 2018, 2019, 2020] (147 échantillons)
   • Test: années [2021, 2022, 2023] (63 échantillons)

📊 Distribution des classes (train):
   • Classe 0 (Faible): 14 (9.5%)
   • Classe 1 (Moyenne): 80 (54.4%)
   • Classe 2 (Haute): 53 (36.1%)


## 5. 🚀 Modèle 1 : XGBoost Régression - Prédiction du Score de Substitution

In [7]:
# 5.1 Entraînement du modèle de base
print("🚀 Entraînement du modèle XGBoost Régression...")
print("="*60)

# Modèle de base avec paramètres par défaut
xgb_reg_base = XGBRegressor(
    objective='reg:squarederror',
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    random_state=42,
    n_jobs=-1
)

# Entraînement
xgb_reg_base.fit(X_train_scaled, y_train_reg)

# Prédictions
y_pred_train_reg = xgb_reg_base.predict(X_train_scaled)
y_pred_test_reg = xgb_reg_base.predict(X_test_scaled)

# Métriques d'évaluation
def evaluate_regression(y_true, y_pred, dataset_name):
    """Évalue les performances d'un modèle de régression"""
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    print(f"\n📊 Résultats {dataset_name}:")
    print(f"   • RMSE: {rmse:.4f}")
    print(f"   • MAE:  {mae:.4f}")
    print(f"   • R²:   {r2:.4f}")
    
    return {'rmse': rmse, 'mae': mae, 'r2': r2}

# Évaluation
metrics_train = evaluate_regression(y_train_reg, y_pred_train_reg, "Train")
metrics_test = evaluate_regression(y_test_reg, y_pred_test_reg, "Test")

print("\n✅ Modèle de base entraîné!")

🚀 Entraînement du modèle XGBoost Régression...

📊 Résultats Train:
   • RMSE: 0.0323
   • MAE:  0.0041
   • R²:   1.0000

📊 Résultats Test:
   • RMSE: 8.5521
   • MAE:  2.9383
   • R²:   0.0650

✅ Modèle de base entraîné!


In [8]:
# 5.2 Optimisation des hyperparamètres avec GridSearchCV
print("🔧 Optimisation des hyperparamètres...")
print("="*60)

# Grille de paramètres à tester
param_grid_reg = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

# Cross-validation temporelle
tscv = TimeSeriesSplit(n_splits=3)

# GridSearchCV
xgb_reg_grid = XGBRegressor(
    objective='reg:squarederror',
    random_state=42,
    n_jobs=-1
)

grid_search_reg = GridSearchCV(
    estimator=xgb_reg_grid,
    param_grid=param_grid_reg,
    cv=3,  # Utiliser CV simple pour la rapidité
    scoring='neg_root_mean_squared_error',
    verbose=1,
    n_jobs=-1
)

# Entraînement avec GridSearch
grid_search_reg.fit(X_train_scaled, y_train_reg)

print(f"\n✅ Meilleurs paramètres trouvés:")
for param, value in grid_search_reg.best_params_.items():
    print(f"   • {param}: {value}")
print(f"\n📊 Meilleur score CV (RMSE): {-grid_search_reg.best_score_:.4f}")

🔧 Optimisation des hyperparamètres...
Fitting 3 folds for each of 108 candidates, totalling 324 fits

✅ Meilleurs paramètres trouvés:
   • colsample_bytree: 0.8
   • learning_rate: 0.01
   • max_depth: 5
   • n_estimators: 100
   • subsample: 1.0

📊 Meilleur score CV (RMSE): 10.5132


In [9]:
# 5.3 Évaluation du modèle optimisé
best_xgb_reg = grid_search_reg.best_estimator_

# Prédictions avec le meilleur modèle
y_pred_train_opt = best_xgb_reg.predict(X_train_scaled)
y_pred_test_opt = best_xgb_reg.predict(X_test_scaled)

print("📊 COMPARAISON: Modèle de base vs Modèle optimisé")
print("="*60)

print("\n🔹 Modèle de BASE:")
evaluate_regression(y_test_reg, y_pred_test_reg, "Test")

print("\n🔹 Modèle OPTIMISÉ:")
metrics_opt = evaluate_regression(y_test_reg, y_pred_test_opt, "Test")

# Visualisation des prédictions vs réalité
fig = make_subplots(rows=1, cols=2,
                    subplot_titles=('Prédictions vs Réalité (Test)', 
                                   'Distribution des Erreurs'))

# Scatter plot
fig.add_trace(
    go.Scatter(x=y_test_reg, y=y_pred_test_opt, mode='markers',
               marker=dict(color='steelblue', size=8, opacity=0.7),
               name='Prédictions'),
    row=1, col=1
)
# Ligne parfaite
fig.add_trace(
    go.Scatter(x=[0, 100], y=[0, 100], mode='lines',
               line=dict(color='red', dash='dash'),
               name='Parfait'),
    row=1, col=1
)

# Histogramme des erreurs
errors = y_test_reg - y_pred_test_opt
fig.add_trace(
    go.Histogram(x=errors, nbinsx=20, marker_color='steelblue', name='Erreurs'),
    row=1, col=2
)

fig.update_layout(
    title_text="📊 Évaluation du Modèle XGBoost Régression",
    height=400,
    showlegend=False
)
fig.update_xaxes(title_text="Valeurs réelles", row=1, col=1)
fig.update_yaxes(title_text="Prédictions", row=1, col=1)
fig.update_xaxes(title_text="Erreur de prédiction", row=1, col=2)
fig.show()

📊 COMPARAISON: Modèle de base vs Modèle optimisé

🔹 Modèle de BASE:

📊 Résultats Test:
   • RMSE: 8.5521
   • MAE:  2.9383
   • R²:   0.0650

🔹 Modèle OPTIMISÉ:

📊 Résultats Test:
   • RMSE: 3.6252
   • MAE:  1.8506
   • R²:   0.8320


## 6. 🎯 Modèle 2 : XGBoost Classification - Classification des Opportunités

In [10]:
# 6.1 Entraînement du modèle de classification
print("🚀 Entraînement du modèle XGBoost Classification...")
print("="*60)

# Modèle de classification multiclasse
xgb_clf = XGBClassifier(
    objective='multi:softmax',
    num_class=3,
    n_estimators=100,
    max_depth=5,
    learning_rate=0.1,
    random_state=42,
    n_jobs=-1,
    use_label_encoder=False,
    eval_metric='mlogloss'
)

# Entraînement
xgb_clf.fit(X_train_scaled, y_train_clf)

# Prédictions
y_pred_train_clf = xgb_clf.predict(X_train_scaled)
y_pred_test_clf = xgb_clf.predict(X_test_scaled)

# Métriques
print("\n📊 Résultats sur le jeu de TEST:")
print(f"   • Accuracy: {accuracy_score(y_test_clf, y_pred_test_clf):.4f}")
print(f"   • Precision (macro): {precision_score(y_test_clf, y_pred_test_clf, average='macro'):.4f}")
print(f"   • Recall (macro): {recall_score(y_test_clf, y_pred_test_clf, average='macro'):.4f}")
print(f"   • F1-Score (macro): {f1_score(y_test_clf, y_pred_test_clf, average='macro'):.4f}")

print("\n📋 Rapport de classification détaillé:")
print(classification_report(y_test_clf, y_pred_test_clf, 
                           target_names=['Faible', 'Moyenne', 'Haute']))

🚀 Entraînement du modèle XGBoost Classification...

📊 Résultats sur le jeu de TEST:
   • Accuracy: 0.9841
   • Precision (macro): 0.9889
   • Recall (macro): 0.9896
   • F1-Score (macro): 0.9891

📋 Rapport de classification détaillé:
              precision    recall  f1-score   support

      Faible       1.00      1.00      1.00         2
     Moyenne       0.97      1.00      0.98        29
       Haute       1.00      0.97      0.98        32

    accuracy                           0.98        63
   macro avg       0.99      0.99      0.99        63
weighted avg       0.98      0.98      0.98        63



In [11]:
# 6.2 Optimisation du modèle de classification
print("🔧 Optimisation des hyperparamètres pour la classification...")
print("="*60)

param_grid_clf = {
    'n_estimators': [50, 100, 150],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.05, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'scale_pos_weight': [1, 2]  # Pour gérer le déséquilibre des classes
}

xgb_clf_grid = XGBClassifier(
    objective='multi:softmax',
    num_class=3,
    random_state=42,
    n_jobs=-1,
    use_label_encoder=False,
    eval_metric='mlogloss'
)

grid_search_clf = GridSearchCV(
    estimator=xgb_clf_grid,
    param_grid=param_grid_clf,
    cv=3,
    scoring='f1_macro',
    verbose=1,
    n_jobs=-1
)

grid_search_clf.fit(X_train_scaled, y_train_clf)

print(f"\n✅ Meilleurs paramètres trouvés:")
for param, value in grid_search_clf.best_params_.items():
    print(f"   • {param}: {value}")
print(f"\n📊 Meilleur score CV (F1-macro): {grid_search_clf.best_score_:.4f}")

🔧 Optimisation des hyperparamètres pour la classification...
Fitting 3 folds for each of 108 candidates, totalling 324 fits

✅ Meilleurs paramètres trouvés:
   • learning_rate: 0.05
   • max_depth: 3
   • n_estimators: 50
   • scale_pos_weight: 1
   • subsample: 1.0

📊 Meilleur score CV (F1-macro): 0.9290


In [12]:
# 6.3 Évaluation du meilleur modèle de classification
best_xgb_clf = grid_search_clf.best_estimator_

y_pred_clf_opt = best_xgb_clf.predict(X_test_scaled)

print("📊 RÉSULTATS DU MODÈLE OPTIMISÉ:")
print("="*60)
print(f"\n   • Accuracy: {accuracy_score(y_test_clf, y_pred_clf_opt):.4f}")
print(f"   • F1-Score (macro): {f1_score(y_test_clf, y_pred_clf_opt, average='macro'):.4f}")

# Matrice de confusion
cm = confusion_matrix(y_test_clf, y_pred_clf_opt)

fig = go.Figure(data=go.Heatmap(
    z=cm,
    x=['Faible', 'Moyenne', 'Haute'],
    y=['Faible', 'Moyenne', 'Haute'],
    colorscale='Blues',
    text=cm,
    texttemplate="%{text}",
    textfont={"size": 16}
))

fig.update_layout(
    title="🎯 Matrice de Confusion - Classification des Opportunités",
    xaxis_title="Classe Prédite",
    yaxis_title="Classe Réelle",
    height=400
)
fig.show()

print("\n📋 Rapport de classification (modèle optimisé):")
print(classification_report(y_test_clf, y_pred_clf_opt,
                           target_names=['Faible', 'Moyenne', 'Haute']))

📊 RÉSULTATS DU MODÈLE OPTIMISÉ:

   • Accuracy: 0.9841
   • F1-Score (macro): 0.9891



📋 Rapport de classification (modèle optimisé):
              precision    recall  f1-score   support

      Faible       1.00      1.00      1.00         2
     Moyenne       0.97      1.00      0.98        29
       Haute       1.00      0.97      0.98        32

    accuracy                           0.98        63
   macro avg       0.99      0.99      0.99        63
weighted avg       0.98      0.98      0.98        63



## 7. 📊 Analyse de l'importance des Features

In [13]:
# 7.1 Importance des features pour le modèle de régression
importance_reg = pd.DataFrame({
    'feature': feature_cols_regression,
    'importance': best_xgb_reg.feature_importances_
}).sort_values('importance', ascending=True)

# Top 15 features
top_features_reg = importance_reg.tail(15)

fig = go.Figure(go.Bar(
    x=top_features_reg['importance'],
    y=top_features_reg['feature'],
    orientation='h',
    marker_color='steelblue'
))

fig.update_layout(
    title="📊 Top 15 Features - Modèle de Régression (Score de Substitution)",
    xaxis_title="Importance",
    yaxis_title="Feature",
    height=500
)
fig.show()

print("📋 Top 10 features les plus importantes (Régression):")
for i, row in importance_reg.tail(10).iloc[::-1].iterrows():
    print(f"   {row['importance']:.4f} - {row['feature']}")

📋 Top 10 features les plus importantes (Régression):
   0.4383 - prix_unitaire_production
   0.1959 - consommation_fcfa
   0.1820 - exports_fcfa
   0.1532 - balance_commerciale_fcfa
   0.0158 - taux_couverture
   0.0046 - ratio_prod_conso_fcfa
   0.0039 - intensite_export
   0.0033 - production_fcfa
   0.0025 - year
   0.0005 - production_tonnes_lag1


In [14]:
# 7.2 Importance des features pour le modèle de classification
importance_clf = pd.DataFrame({
    'feature': feature_cols_classification,
    'importance': best_xgb_clf.feature_importances_
}).sort_values('importance', ascending=True)

# Top 15 features
top_features_clf = importance_clf.tail(15)

fig = go.Figure(go.Bar(
    x=top_features_clf['importance'],
    y=top_features_clf['feature'],
    orientation='h',
    marker_color='#27ae60'
))

fig.update_layout(
    title="📊 Top 15 Features - Modèle de Classification (Opportunités)",
    xaxis_title="Importance",
    yaxis_title="Feature",
    height=500
)
fig.show()

print("📋 Top 10 features les plus importantes (Classification):")
for i, row in importance_clf.tail(10).iloc[::-1].iterrows():
    print(f"   {row['importance']:.4f} - {row['feature']}")

📋 Top 10 features les plus importantes (Classification):
   0.3336 - production_fcfa_growth
   0.1582 - taux_couverture
   0.1435 - consommation_fcfa
   0.1066 - ratio_prod_conso_fcfa
   0.0726 - exports_fcfa
   0.0508 - imports_fcfa_growth
   0.0387 - intensite_export
   0.0203 - production_tonnes_growth
   0.0128 - production_fcfa
   0.0128 - production_fcfa_lag2


## 8. 🔍 Analyse des prédictions par secteur

In [15]:
# Ajouter les prédictions au dataset de test
df_test = df_ml[test_mask].copy()
df_test['predicted_score'] = y_pred_test_opt
df_test['predicted_class'] = y_pred_clf_opt
df_test['predicted_label'] = df_test['predicted_class'].map(opportunity_labels)

# Analyser les opportunités par secteur (moyenne sur les années de test)
sector_analysis = df_test.groupby('LIBELLES').agg({
    'predicted_score': 'mean',
    'substitution_score_normalized': 'mean',
    'predicted_class': lambda x: x.mode()[0],  # Classe la plus fréquente
    'production_fcfa': 'mean',
    'imports_fcfa': 'mean'
}).round(2)

sector_analysis['predicted_label'] = sector_analysis['predicted_class'].map(opportunity_labels)
sector_analysis = sector_analysis.sort_values('predicted_score', ascending=False)

print("📊 ANALYSE DES OPPORTUNITÉS DE SUBSTITUTION PAR SECTEUR")
print("="*80)
print("\n🏆 Top 10 secteurs avec le plus fort potentiel de substitution:")
print("-"*80)

for i, (sector, row) in enumerate(sector_analysis.head(10).iterrows(), 1):
    print(f"\n{i:2d}. {sector[:60]}...")
    print(f"    • Score prédit: {row['predicted_score']:.1f}/100")
    print(f"    • Classe: {row['predicted_label']}")
    print(f"    • Production moyenne: {row['production_fcfa']:.1f} Mds FCFA")
    print(f"    • Imports moyens: {row['imports_fcfa']:.1f} Mds FCFA")

📊 ANALYSE DES OPPORTUNITÉS DE SUBSTITUTION PAR SECTEUR

🏆 Top 10 secteurs avec le plus fort potentiel de substitution:
--------------------------------------------------------------------------------

 1. Perles fines ou de culture, pierres gemmes ou similaires, mé...
    • Score prédit: 29.2/100
    • Classe: Haute
    • Production moyenne: 2138.1 Mds FCFA
    • Imports moyens: 0.8 Mds FCFA

 2. Matières textiles et ouvrages en ces matières...
    • Score prédit: 7.2/100
    • Classe: Haute
    • Production moyenne: 286.2 Mds FCFA
    • Imports moyens: 48.4 Mds FCFA

 3. Bois, charbons de bois et ouvrages en bois...
    • Score prédit: 1.1/100
    • Classe: Haute
    • Production moyenne: 8.1 Mds FCFA
    • Imports moyens: 7.8 Mds FCFA

 4. Armes, munitions et leurs parties et accessoires...
    • Score prédit: 1.1/100
    • Classe: Moyenne
    • Production moyenne: 1.4 Mds FCFA
    • Imports moyens: 1.4 Mds FCFA

 5. Animaux vivants et produits du règne animal...
    • Score prédit: 

In [16]:
# Visualisation des opportunités par secteur
fig = px.bar(
    sector_analysis.reset_index().head(15),
    x='predicted_score',
    y='LIBELLES',
    color='predicted_label',
    color_discrete_map={'Haute': '#27ae60', 'Moyenne': '#f39c12', 'Faible': '#e74c3c'},
    orientation='h',
    title="🎯 Score de Substitution par Secteur (Top 15)"
)

fig.update_layout(
    xaxis_title="Score de Substitution Prédit",
    yaxis_title="Secteur",
    height=600,
    yaxis={'categoryorder': 'total ascending'}
)
fig.show()

## 9. 💾 Sauvegarde des modèles entraînés

In [18]:
# Sauvegarder les modèles
print("💾 Sauvegarde des modèles...")
print("="*60)

# Modèle de régression (utiliser pickle pour compatibilité)
with open(MODELS_PATH / "xgb_regression_substitution.pkl", 'wb') as f:
    pickle.dump(best_xgb_reg, f)
print(f"✅ Modèle de régression sauvegardé: xgb_regression_substitution.pkl")

# Modèle de classification
with open(MODELS_PATH / "xgb_classification_opportunity.pkl", 'wb') as f:
    pickle.dump(best_xgb_clf, f)
print(f"✅ Modèle de classification sauvegardé: xgb_classification_opportunity.pkl")

# Sauvegarder le scaler
with open(MODELS_PATH / "scaler.pkl", 'wb') as f:
    pickle.dump(scaler, f)
print(f"✅ Scaler sauvegardé: scaler.pkl")

# Sauvegarder les métadonnées des modèles
model_metadata = {
    'date_training': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S'),
    'regression_model': {
        'name': 'XGBoost Regressor',
        'target': 'substitution_score_normalized',
        'best_params': grid_search_reg.best_params_,
        'metrics': {
            'rmse': metrics_opt['rmse'],
            'mae': metrics_opt['mae'],
            'r2': metrics_opt['r2']
        }
    },
    'classification_model': {
        'name': 'XGBoost Classifier',
        'target': 'opportunity_class',
        'classes': opportunity_labels,
        'best_params': grid_search_clf.best_params_,
        'metrics': {
            'accuracy': accuracy_score(y_test_clf, y_pred_clf_opt),
            'f1_macro': f1_score(y_test_clf, y_pred_clf_opt, average='macro')
        }
    },
    'feature_columns': feature_cols_regression,
    'train_years': train_years,
    'test_years': test_years,
    'feature_importance_regression': importance_reg.set_index('feature')['importance'].to_dict(),
    'feature_importance_classification': importance_clf.set_index('feature')['importance'].to_dict()
}

with open(MODELS_PATH / "model_metadata.json", 'w', encoding='utf-8') as f:
    json.dump(model_metadata, f, ensure_ascii=False, indent=2, default=str)
print(f"✅ Métadonnées sauvegardées: model_metadata.json")

# Sauvegarder l'analyse par secteur
sector_analysis.to_csv(MODELS_PATH / "sector_opportunities.csv")
print(f"✅ Analyse sectorielle sauvegardée: sector_opportunities.csv")

print(f"\n📁 Tous les fichiers sont dans: {MODELS_PATH.absolute()}")

💾 Sauvegarde des modèles...
✅ Modèle de régression sauvegardé: xgb_regression_substitution.pkl
✅ Modèle de classification sauvegardé: xgb_classification_opportunity.pkl
✅ Scaler sauvegardé: scaler.pkl
✅ Métadonnées sauvegardées: model_metadata.json
✅ Analyse sectorielle sauvegardée: sector_opportunities.csv

📁 Tous les fichiers sont dans: c:\hackathon-24h\notebooks\..\models


## 10. 📝 Résumé et conclusions

In [19]:
# Résumé final
print("="*70)
print("📊 RÉSUMÉ DE L'ENTRAÎNEMENT DES MODÈLES XGBOOST")
print("="*70)

print(f"""
✅ MODÈLES ENTRAÎNÉS AVEC SUCCÈS!

🎯 MODÈLE 1 : XGBoost Régression
   ─────────────────────────────
   • Objectif: Prédire le score de substitution aux importations
   • Meilleurs paramètres: {grid_search_reg.best_params_}
   • Performance (Test):
     - RMSE: {metrics_opt['rmse']:.4f}
     - MAE:  {metrics_opt['mae']:.4f}
     - R²:   {metrics_opt['r2']:.4f}

🎯 MODÈLE 2 : XGBoost Classification
   ───────────────────────────────────
   • Objectif: Classifier les opportunités (Faible/Moyenne/Haute)
   • Meilleurs paramètres: {grid_search_clf.best_params_}
   • Performance (Test):
     - Accuracy:  {accuracy_score(y_test_clf, y_pred_clf_opt):.4f}
     - F1-Score:  {f1_score(y_test_clf, y_pred_clf_opt, average='macro'):.4f}

📊 FEATURES LES PLUS IMPORTANTES:
   1. {importance_reg.tail(1)['feature'].values[0]}
   2. {importance_reg.tail(2).head(1)['feature'].values[0]}
   3. {importance_reg.tail(3).head(1)['feature'].values[0]}

🏆 SECTEURS À FORT POTENTIEL DE SUBSTITUTION:
""")

for i, (sector, row) in enumerate(sector_analysis.head(5).iterrows(), 1):
    print(f"   {i}. {sector[:50]}... (Score: {row['predicted_score']:.1f})")

print(f"""
📁 FICHIERS GÉNÉRÉS:
   • xgb_regression_substitution.pkl
   • xgb_classification_opportunity.pkl
   • scaler.pkl
   • model_metadata.json
   • sector_opportunities.csv

🎯 Prêt pour le Notebook 4: Évaluation et Visualisation!
""")
print("="*70)

📊 RÉSUMÉ DE L'ENTRAÎNEMENT DES MODÈLES XGBOOST

✅ MODÈLES ENTRAÎNÉS AVEC SUCCÈS!

🎯 MODÈLE 1 : XGBoost Régression
   ─────────────────────────────
   • Objectif: Prédire le score de substitution aux importations
   • Meilleurs paramètres: {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 100, 'subsample': 1.0}
   • Performance (Test):
     - RMSE: 3.6252
     - MAE:  1.8506
     - R²:   0.8320

🎯 MODÈLE 2 : XGBoost Classification
   ───────────────────────────────────
   • Objectif: Classifier les opportunités (Faible/Moyenne/Haute)
   • Meilleurs paramètres: {'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 50, 'scale_pos_weight': 1, 'subsample': 1.0}
   • Performance (Test):
     - Accuracy:  0.9841
     - F1-Score:  0.9891

📊 FEATURES LES PLUS IMPORTANTES:
   1. prix_unitaire_production
   2. consommation_fcfa
   3. exports_fcfa

🏆 SECTEURS À FORT POTENTIEL DE SUBSTITUTION:

   1. Perles fines ou de culture, pierres gemmes ou simi... (Score: 29.2